# Libraries

* pypdf 
* langchain 
* Chroma

In [1]:
#!curl -o paper.pdf https://arxiv.org/pdf/2311.06517.pdf

# All necessary imports

In [2]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.vectorstores import Chroma 
from langchain.chains import ChatVectorDBChain
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings

# LLM Model

Zephyr is a series of language models that are trained to act as helpful assistants, I chose the Zephyr cause it's only 7B parameters </br>
Source: https://huggingface.co/HuggingFaceH4/zephyr-7b-beta

In [3]:
ollama_model = "mistral"

In [4]:
llm = Ollama(
    model=ollama_model, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# PDF Loader

Langchain has several document loaders, is a type of tool that loads types of files, like csv, 

In [5]:
loader = PyPDFLoader("paper.pdf")
pages = loader.load_and_split()
len(pages)

26

# Embeddings

I chose zephyr as my LLM model, so I use the same model to crate the embedding. </br>
Embeddings refer to a technique in machine learning and natural language processing where words, phrases, or entities are represented as vectors of real numbers in a multi-dimensional space. The key idea behind embeddings is to capture semantic relationships between words or entities, such that similar words or entities are mapped to nearby points in the vector space.

In [6]:
embeddings = OllamaEmbeddings(model=ollama_model)

# Persist the db in the directory

with `Chroma.from_documents` we can create the vectorstore or ai database  </br>
After that you'll have the database in disk, to use again you can load from disk: `Chroma(persist_directory='database_path')`

In [7]:
%%time
vector_db = Chroma.from_documents(pages, embedding=embeddings , persist_directory='./vector_store')
vector_db.persist()

KeyboardInterrupt: 

# Ask questions to PDF/VectorDB

In [8]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_db.as_retriever())

NameError: name 'vector_db' is not defined

In [ ]:
question = "what is about the article?"
result = qa_chain({"query": question})

In [ ]:
result

# Ask with context

the same way you talk with a chatbot using llm, they will remember all the context of the conversation, below, there is a simple example.

In [ ]:
question = "Make a summary about the BClean using simple words"
result = qa_chain({"query": question})

In [ ]:
context = []
context.append({"question": result['query'], "answer": result["result"]})

In [ ]:
#context = f"Question: {result['query']} | Answer: {result['result']}"
context

In [ ]:
question = "Make a summary in portuguese"
result = qa_chain({"context": context , "query": question})

In [ ]:
context.append({"question": result['query'], "answer": result["result"]})

In [ ]:
question
result = qa_chain({"context": context , "query": question})

In [ ]:
qa_chain2 = RetrievalQAWithSourcesChain.from_chain_type(llm, retriever=vector_db.as_retriever(),  return_source_documents=True)

In [ ]:
question = "Make a summary about the BClean using simple words, show as a solution"
result = qa_chain2({"question": question})

In [ ]:
result

In [ ]:
context = f"Question: {result['question']} | Answer: {result['answer']}"
result = qa_chain2({"context": context , "question": "Make a longer summary"})

In [ ]:
result

# Load Multiple Documents (Only Example)

```python
loader1 = PyPDFLoader("document1.pdf")
pages1 = loader1.load_and_split()

loader2 = PyPDFLoader("document2.pdf")
pages2 = loader2.load_and_split()

embeddings = OllamaEmbeddings(model=ollama_model)

vector_db = Chroma.from_documents(pages1 + pages2, embedding=embeddings, persist_directory='.')
vector_db.persist()
```